In [1]:
#import mvpa2
import pandas as pd
import numpy as np
#import sklearn
#to manipulate data
from mvpa2.datasets import *
import scipy.io

from mvpa2.suite import *

if __debug__:
    debug.active.append('SMLR_')
    
import matplotlib as plt
%matplotlib inline

/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/base/hdf5.py:40: H5pyDeprecationWarning: The h5py.highlevel module is deprecated, code should import directly from h5py, e.g. 'from h5py import File'.
  import h5py.highlevel  # >= 2.8.0, https://github.com/h5py/h5py/issues/1063
/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/pydicom/__init__.py:55: DeprecationWarning: Python 2 will no longer be supported after the pydicom v1.4 release
  warnings.warn(msg, DeprecationWarning)
/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/testing/tools.py:81: DeprecationWarning: Importing from numpy.testing.decorators is deprecated since numpy 1.15.0, import from numpy.testing instead.
  from numpy.testing.decorators import skipif


In [2]:
#makes a dict for task
matFile = scipy.io.loadmat('/Users/aporter1350/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/mem/MSC02_parcel_corrmat.mat')
#convert to numpy array
task_FC=np.array(matFile['parcel_corrmat'])
#Index of upper triangle of matrix 
nrois=333
nsess=10
task_mask=np.triu_indices(nrois, 1)
task_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=task_FC[:,:,sess]
    task_corrlin[count]=tmp[task_mask]
    count=count+1

#makes a dict for rest
matFile = scipy.io.loadmat('/Users/aporter1350/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/MSC02_parcel_corrmat.mat')
#convert to numpy array
rest_FC=np.array(matFile['parcel_corrmat'])
#Index of upper triangle of matrix 
nrois=333
nsess=10
rest_mask=np.triu_indices(nrois, 1)
rest_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=rest_FC[:,:,sess]
    rest_corrlin[count]=tmp[rest_mask]
    count=count+1
    
#makes a dict for test task
matFile = scipy.io.loadmat('/Users/aporter1350/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/motor/MSC02_parcel_corrmat.mat')
#convert to numpy array
motor_FC=np.array(matFile['parcel_corrmat'])
#Index of upper triangle of matrix 
nrois=333
nsess=10
test_mask=np.triu_indices(nrois, 1)
motor_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=motor_FC[:,:,sess]
    motor_corrlin[count]=tmp[test_mask]
    count=count+1
#corrlin.shape
#task_ds=Dataset(corrlin)
#ds.shape
#ds.nfeatures
#ds.nsamples
#create a training dataset from the labeled features
task_ds=dataset_wizard(samples=task_corrlin, targets=1)
rest_ds=dataset_wizard(samples=rest_corrlin, targets=0)
trainpat=vstack((task_ds, rest_ds))

#create pattern for testing dataset
test_ds=dataset_wizard(samples=motor_corrlin, targets=1)

In [5]:
#setup SMLR classifier
print "evaluating SMLR classifier"
smlr=SMLR(fit_all_weights=True)
#enable saving of estimates used for prediction
smlr.ca.enable('estimates')
#train with the known points
smlr.train(trainpat)
#run predictions on test values
pre=smlr.predict(test_ds.samples)
#caluclate confusion matrix
smlr_confusion=ConfusionMatrix(labels=trainpat.UT, targets=test_ds.targets, predictions=pre)

evaluating SMLR classifier
[SMLR_] DBG:                     hstacking 1s for bias
[SMLR_] DBG:                     Calling stepwise_regression. Seed 331701662


In [6]:
#linear SVM
print "evaluating linear SVM classifier"
lsvm=LinearNuSVMC(probability=1)
#save estimates
lsvm.ca.enable('estimates')
#train with known points
lsvm.train(trainpat)
#run predictions on test values
pre=lsvm.predict(test_ds.samples)
#calculate confusion matrix
lsvm_confusion=ConfusionMatrix(labels=trainpat.UT, targets=test_ds.targets,predictions=pre)

evaluating linear SVM classifier


/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/misc/attrmap.py:153: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype(str).type`.
  if not np.issubdtype(attr.dtype, str) and not self.mapnumeric:
/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:215: FutureWarning: Conversion of the second argument of issubdtype from `'c'` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype('c').type`.
  if ( np.issubdtype(self.ca.trained_targets.dtype, 'c') or
/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:216: FutureWarning: Conversion of the second argument of issubdtype from `'U'` to `unicode` is deprecated. In future, it will be treated as `np.unicode_ == np.dtype('U').type`.
  np.issubdtype(self.ca.trained_targets.dtype, 'U') ):


In [7]:
#train SWM with selected features
print "evaluating linear SVM classifier with SMLR features"
keepInd=(np.abs(smlr.weights).mean(axis=1)!=0)
newtrainpat=trainpat[:, keepInd]
newtestpat=test_ds[:, keepInd]
#train with known points
lsvm.train(newtrainpat)
#run prediction on test values
pre=lsvm.predict(newtestpat.samples)
#calculate confusion matrix
lsvm_confusion_sparse=ConfusionMatrix(labels=newtrainpat.UT, targets=newtestpat.targets, predictions=pre)


evaluating linear SVM classifier with SMLR features


In [8]:
print "SMLR Percent Correct:\t%g%% (Retained %d/%d features)" % \
    (smlr_confusion.percent_correct,
     (smlr.weights!=0).sum(), np.prod(smlr.weights.shape))
print "linear-SVM Percent Correct:\t%g%%" % \
    (lsvm_confusion.percent_correct)
print "linear-SVM Percent Correct (with %d features from SMLR):\t%g%%" % \
    (keepInd.sum(), lsvm_confusion_sparse.percent_correct)

SMLR Percent Correct:	60% (Retained 36/110556 features)
linear-SVM Percent Correct:	30%
linear-SVM Percent Correct (with 18 features from SMLR):	60%


/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/transerror.py:681: RuntimeWarning: invalid value encountered in divide
  stats['TPR'] = stats['TP'] / (1.0*stats['P'])
/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/transerror.py:688: RuntimeWarning: invalid value encountered in divide
  stats['SPC'] = (stats['TN']) / (1.0*stats['FP'] + stats['TN'])


In [9]:
lsvm_confusion_sparse.matrix

array([[0, 4],
       [0, 6]])

In [3]:
#linear SVM
print "evaluating linear SVM classifier"
lsvm=LinearNuSVMC(probability=1)
#save estimates
lsvm.ca.enable('estimates')
#train with known points
lsvm.train(trainpat)
#run predictions on test values
pre=lsvm.predict(trainpat.samples)
np.mean(pre==trainpat.sa.targets)

evaluating linear SVM classifier


/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/misc/attrmap.py:153: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype(str).type`.
  if not np.issubdtype(attr.dtype, str) and not self.mapnumeric:
/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:215: FutureWarning: Conversion of the second argument of issubdtype from `'c'` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype('c').type`.
  if ( np.issubdtype(self.ca.trained_targets.dtype, 'c') or
/Applications/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:216: FutureWarning: Conversion of the second argument of issubdtype from `'U'` to `unicode` is deprecated. In future, it will be treated as `np.unicode_ == np.dtype('U').type`.
  np.issubdtype(self.ca.trained_targets.dtype, 'U') ):


1.0